# SONAR-netCDF4 Convention checker

6/12,8,7,6,5,1/2023

see https://github.com/OSOceanAcoustics/echopype/issues/1043


**TODO:**
- Provide an option in the tests to return a list (or other appropriate data structures) instead of doing print out

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys

import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr

import echopype as ep
from echopype.testing import TEST_DATA_FOLDER
# TEST_DATA_FOLDER = Path("/usr/mayorgadat/workmain/acoustics/gh/OSOceanAcoustics/echopype/echopype/test_data/")

# sys.path.append("/usr/mayorgadat/workmain/acoustics/gh/OSOceanAcoustics/convention-checker/convention_checker/")
from convention_checker import cc

ep.__version__

'0.6.4.dev132+g439819b3.d20230609'

## Test reading and parsing a CDL file

In [3]:
target_group = "Platform" # "Sonar/Beam_group1"

### Read and process the CDL

In [4]:
conv_check = cc.ConventionCDL(target_group)

In [5]:
conv_check.cdl_ds

<xarray.Dataset>
Dimensions:                      (channel: 3, time1: 1, time2: 5)
Coordinates:
  * channel                      (channel) object 'WBT 150013-15 ES120-7C_ES'...
  * time1                        (time1) datetime64[ns] 2021-10-05T00:16:12
  * time2                        (time2) datetime64[ns] 2021-10-05T00:16:12 ....
Data variables: (12/23)
    drop_keel_offset             float64 nan
    drop_keel_offset_is_manual   int64 0
    frequency_nominal            (channel) float64 1.2e+05 2e+05 7e+04
    latitude                     (time1) float64 50.0
    longitude                    (time1) float64 -130.1
    MRU_offset_x                 float32 nan
    ...                           ...
    transducer_offset_x          (channel) float32 0.0 0.0 0.0
    transducer_offset_y          (channel) float32 0.0 0.0 0.0
    transducer_offset_z          (channel) float32 0.0 0.0 0.0
    vertical_offset              (time2) float32 0.0 0.0 0.0 0.0 0.0
    water_level                  float32 0.0
    water_level_draft_is_manual  int64 0
Attributes:
    platform_code_ICES:  
    platform_name:       
    platform_type:

In [6]:
conv_check.cdl_variables_df

,variable_name,obligation,echopype_mods,data_type,long_name,units,standard_name,valid_min,valid_range,comment
0,MRU_offset_x,R,None,float32,Distance along the x-axis from the platform co...,m,None,NaN,None,None
1,MRU_offset_y,R,None,float32,Distance along the y-axis from the platform co...,m,None,NaN,None,None
2,MRU_offset_z,R,None,float32,Distance along the z-axis from the platform co...,m,None,NaN,None,None
3,MRU_rotation_x,R,None,float32,Extrinsic rotation about the x-axis from the p...,arc_degree,None,NaN,"[-180.0, 180.0]",None
4,MRU_rotation_y,R,None,float32,Extrinsic rotation about the y-axis from the p...,arc_degree,None,NaN,"[-180.0, 180.0]",None
5,MRU_rotation_z,R,None,float32,Extrinsic rotation about the z-axis from the p...,arc_degree,None,NaN,"[-180.0, 180.0]",None
6,channel,NA,added,object,Vendor channel ID,None,None,NaN,None,None
7,drop_keel_offset,NA,added,float64,None,None,None,NaN,None,None
8,drop_keel_offset_is_manual,NA,added,int64,None,None,None,NaN,None,None
9,frequency_nominal,NA,added,float64,Transducer frequency,Hz,sound_frequency,0.0,None,None


In [7]:
conv_check.cdl_variables_df.echopype_mods.value_counts(dropna=False)

None             18
added             6
changed-units     2
Name: echopype_mods, dtype: int64

In [8]:
conv_check.cdl_variables_df.obligation.value_counts(dropna=False)

R     14
NA     8
MA     4
Name: obligation, dtype: int64

In [9]:
list(conv_check.cdl_ds.coords)

['channel', 'time1', 'time2']

In [10]:
# type(cdl_ds['sample_interval'].attrs['valid_min'])

### First tests against a converted EchoData object

In [11]:
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80_bb_with_calibration/2018115-D20181213-T094600.raw", sonar_model='EK80')
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/Summer2018--D20180905-T033113.raw", sonar_model='EK80')
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/D20170912-T234910.raw", sonar_model='EK80')
ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/2019118 group2survey-D20191214-T081342.raw", sonar_model='EK80')
# waveform_mode, encode_mode = "CW", "power"

# ed = ep.open_raw(TEST_DATA_FOLDER / "ek60/ooi/CE02SHBP-MJ01C-07-ZPLSCB101_OOI-D20191201-T000000.raw", sonar_model='EK60')
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek60/ncei-wcsd/Summer2017-D20170620-T011027.raw", sonar_model='EK60')
# waveform_mode, encode_mode = None, None

# Rutgers glider data, with empty tilt_x and tilt_y
# ed = ep.open_raw(
#     TEST_DATA_FOLDER / 'azfp/rutgers_glider_notemperature/18011107.01A', 
#     xml_path=TEST_DATA_FOLDER / 'azfp/rutgers_glider_notemperature/18011107.XML', 
#     sonar_model='AZFP'
# )
# A more normal data, with valid tilt_x and tilt_y
# ed = ep.open_raw(
#     TEST_DATA_FOLDER / 'azfp/17082117.01A', 
#     xml_path=TEST_DATA_FOLDER / 'azfp/17041823.XML', 
#     sonar_model='AZFP'
# )
# waveform_mode, encode_mode = None, None

In [12]:
ed['Platform']

<xarray.Dataset>
Dimensions:                      (time1: 3, time2: 1, channel: 2)
Coordinates:
  * channel                      (channel) <U22 'WBT 714581-15 ES18' 'WBT 714...
  * time1                        (time1) datetime64[ns] 2019-12-14T08:13:44.6...
  * time2                        (time2) datetime64[ns] 2019-12-14T08:13:44
Data variables: (12/23)
    latitude                     (time1) float64 60.43 60.43 60.43
    longitude                    (time1) float64 5.528 5.528 5.528
    sentence_type                (time1) <U3 'GGA' 'GLL' 'RMC'
    pitch                        (time2) float64 -0.06
    roll                         (time2) float64 0.03
    vertical_offset              (time2) float64 0.0
    ...                           ...
    MRU_rotation_y               float64 nan
    MRU_rotation_z               float64 nan
    position_offset_x            float64 nan
    position_offset_y            float64 nan
    position_offset_z            float64 nan
    frequency_nominal            (channel) float64 1.8e+04 3.33e+05
Attributes:
    platform_name:       
    platform_type:       
    platform_code_ICES:

In [13]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains complex backscatter data and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

In [14]:
ed['Platform'].vertical_offset.min().values, ed['Platform'].vertical_offset.max().values

(array(0.), array(0.))

In [15]:
ed['Platform'].water_level.min().values, ed['Platform'].water_level.max().values

(array(0.), array(0.))

In [16]:
if ed.sonar_model == "EK80":
    ed['Platform'].drop_keel_offset.min().values, ed['Platform'].drop_keel_offset.max().values

In [17]:
ed_group_ds = ed[target_group]

In [18]:
conv_check.set_ed_group_ds(ed_group_ds)

#### Set the obligation and echopype_mods filters (optional)

In [19]:
conv_check.set_obligation(None) # "M", ['M', 'MA'], None, exclude=True

In [20]:
conv_check._get_obligation_vars()

array(['MRU_offset_x', 'MRU_offset_y', 'MRU_offset_z', 'MRU_rotation_x',
       'MRU_rotation_y', 'MRU_rotation_z', 'channel', 'drop_keel_offset',
       'drop_keel_offset_is_manual', 'frequency_nominal', 'latitude',
       'longitude', 'pitch', 'position_offset_x', 'position_offset_y',
       'position_offset_z', 'roll', 'sentence_type', 'time1', 'time2',
       'transducer_offset_x', 'transducer_offset_y',
       'transducer_offset_z', 'vertical_offset', 'water_level',
       'water_level_draft_is_manual'], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [21]:
conv_check.set_echopype_mods(None) # None

In [22]:
conv_check.echopype_mods

#### Presence of expected variables

In [23]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:


#### Presence of unexpected variables

In [24]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [25]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
MRU_rotation_x: EchoData type: float64, CDL type: float32
MRU_offset_x: EchoData type: float64, CDL type: float32
water_level: EchoData type: float64, CDL type: float32
sentence_type: EchoData type: <U3, CDL type: object
transducer_offset_y: EchoData type: float64, CDL type: float32
position_offset_y: EchoData type: float64, CDL type: float32
transducer_offset_z: EchoData type: float64, CDL type: float32
MRU_offset_y: EchoData type: float64, CDL type: float32
position_offset_x: EchoData type: float64, CDL type: float32
MRU_rotation_z: EchoData type: float64, CDL type: float32
roll: EchoData type: float64, CDL type: float32
channel: EchoData type: <U22, CDL type: object
MRU_offset_z: EchoData type: float64, CDL type: float32
transducer_offset_x: EchoData type: float64, CDL type: float32
position_offset_z: EchoData type: float64, CDL type: float32
MRU_rotation_y: EchoData type: float64, CDL type: float32
vertical_offset: EchoD

In [26]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:
sentence_type: EchoData type: <U3, CDL type: object


#### Variable dimensionality

In [27]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [28]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:


In [29]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

- This will be trickier b/c of values that are not defined as static
- **NOTE:** Currently excluding `valid_range` b/c it entails comparing lists or arrays, not scalars

In [30]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:


In [31]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
